<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_19/1_Saved_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сохранение моделей TensorFlow

В этом уроке мы рассмотрим стандартный способ сохранения и последующей загрузки моделей на TensorFlow. После того, как модель обучена, необходимо сохранить её структуру и обученные веса в файл. Можно это сделать вручную, но более удобные сопособ — с использованием понятия `saved model`. Такое представление хранит в себе как вычислительный граф, так и обученные веса.

Для примера создадим простую свёрточную нейросеть и обучим её на датасете MNIST. Затем сохраним её и загрузим уже в другом ноутбуке (имитируя реальную ситуацию использования модели после обучения).

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.x.

Переключаемся на версию 2.0 (работает только в Colab).

### Загрузка библиотек

In [1]:
import numpy as np
import tensorflow as tf

### Загрузка датасета MNIST

Загрузим MNIST и подготовим его для обучения — сделаем reshape (чтобы привести векторизованные картинки в двумерные) и приведём к float32 в диапазоне [0, 1].

In [2]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

train_x = train_x.reshape(-1, 28, 28, 1).astype(np.float32) / 255.
test_x = test_x.reshape(-1, 28, 28, 1).astype(np.float32) / 255.

11501568/11490434 [==============================] - 0s 0us/step


### Создание модели

Создадим простую свёрточную нейронную сеть для классификации на 10 классов (цифры из MNIST). Сделаем это с помощью `keras.models.Sequential`.

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2), (2, 2)),
    tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2), (2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

### Подготовка модели к обучению

In [4]:
model.compile(optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

### Обучение модели

In [6]:
NUM_EPOCHS = 4
model.fit(train_x, train_y, epochs=NUM_EPOCHS)

1875/1875 [==============================] - 165s 88ms/step - loss: 0.0510 - accuracy: 0.9847


### Оценка качества обученной модели

In [7]:
model.evaluate(test_x, test_y)

313/313 [==============================] - 7s 22ms/step - loss: 0.0425 - accuracy: 0.9856


[0.042493242770433426, 0.9855999946594238]

### Сохранение модели

После того, как наша модель обучена, настало время её сохранить. Так как наша модель была создана с помощью `keras`, сохранение выглядит совсем просто: вызываем метод `save()` и указываем путь, по которому следует сохранить модель. Это будет некая директория с файлами.

Зачастую необходимо хранить информацию о версии модели. В TensorFlow есть соглашение, что версия — это название директории, в которой лежит модель. Пусть, например, у нас будет версия `1` — тогда сохраним модель в директорию `./saved_model/1`.

In [8]:
model.save('./saved_model/1')

Для удобства скачивания директории из Colab предварительно заархивируем всю директорию `saved_model`. Затем мы сможем легко скачать получившийся zip-архив, чтобы затем загрузить его в другой ноутбук или использовать где-то ещё.

In [9]:
!zip -r saved_model.zip saved_model

  adding: saved_model/ (stored 0%)
  adding: saved_model/1/ (stored 0%)
  adding: saved_model/1/saved_model.pb (deflated 88%)
  adding: saved_model/1/assets/ (stored 0%)
  adding: saved_model/1/variables/ (stored 0%)
  adding: saved_model/1/variables/variables.index (deflated 55%)
  adding: saved_model/1/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: saved_model/1/keras_metadata.pb (deflated 90%)
